# Calling Keras layers on TensorFlow tensors

In [13]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [4]:
#input
img = tf.placeholder(tf.float32,shape=(None,784))

#build model using keras in a fast manner
from keras.layers import Dense

x = Dense(128,activation='relu')(img)
x = Dense(128,activation='relu')(x)
preds = Dense(10,activation='softmax')(x)

#output
labels = tf.placeholder(tf.float32,shape=(None,10))

#loss function
from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels,preds))

In [8]:
#train model
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_DATA',one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

#run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img:batch[0],labels:batch[1]})


Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [9]:
#evaluation
from keras.metrics import categorical_accuracy as accuracy

acc_value = accuracy(labels,preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img:mnist_data.test.images,labels:mnist_data.test.labels}))

[1. 1. 1. ... 1. 1. 1.]


In [10]:
#Different behaviors in training and testing phase
K.learning_phase()

<tf.Tensor 'keras_learning_phase:0' shape=() dtype=bool>

In [21]:
from keras.layers import Dropout
from keras import backend as K

img = tf.placeholder(tf.float32,shape=(None,784))
labels = tf.placeholder(tf.float32,shape=(None,10))

x = Dense(128,activation='relu')(img)
x = Dropout(0.5)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(10,activation='softmax')(x)

loss = tf.reduce_mean(categorical_crossentropy(labels,preds))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

#run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img:batch[0],labels:batch[1],K.learning_phase():1})
        
acc_value = accuracy(labels,preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img:mnist_data.test.images,labels:mnist_data.test.labels,K.learning_phase():0}))

[1. 0. 1. ... 0. 1. 1.]


In [22]:
# compatibility with name scope
from keras.layers import LSTM
x = tf.placeholder(tf.float32,shape=(None,20,64))
with tf.name_scope('block1'):
    y = LSTM(32,name='mylstm')(x)
#the weigths of lstm layer will be named block1/mylstm_W_i, block1/mylstm_U_i, etc.

In [24]:
# compatibility with device scope
with tf.device('/cpu:0'):
    x = tf.placeholder(tf.float32,shape=(None,20,64))
    y = LSTM(32,name='mylstm')(x)
#all ops/variables in the lstm layer will be on cpu:0
    

In [25]:
# compatibility with graph scope
my_graph = tf.Graph()
with my_graph.as_default():
    x = tf.placeholder(tf.float32,shape=(None,20,64))
    y = LSTM(32,name='mylstm')(x)
#all ops/variables in the lstm layer are careted as part of graph

In [26]:
# compatibility with variable scopes
lstm = LSTM(32)

x = tf.placeholder(tf.float32,shape=(None,20,64))
y = tf.placeholder(tf.float32,shape=(None,20,64))

x_encoded = lstm(x)
y_encoded = lstm(y)


# Using Keras models with TensorFlow

In [34]:
from keras.layers import InputLayer

# this is our modified Keras model
model = Sequential()
model.add(InputLayer(input_tensor=custom_input_tensor,
                     input_shape=(None, 784)))

# build the rest of the model as before
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

NameError: name 'custom_input_tensor' is not defined

In [29]:
from keras.models import Sequential

model = Sequential()
model.add(Dense(32,activation='relu',input_dim=784))
model.add(Dense(10,activation='softmax'))

x = tf.placeholder(tf.float32,shape=(None,784))
y = model(x)

# Multi-GPU and distributed training

In [30]:
with tf.device('/cpu:0'):
    x = tf.placeholder(tf.float32, shape=(None, 784))

    # shared model living on CPU:0
    # it won't actually be run during training; it acts as an op template
    # and as a repository for shared variables
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=784))
    model.add(Dense(10, activation='softmax'))

# replica 0
with tf.device('/gpu:0'):
    output_0 = model(x)  # all ops in the replica will live on GPU:0

# replica 1
with tf.device('/gpu:1'):
    output_1 = model(x)  # all ops in the replica will live on GPU:1

# merge outputs on CPU
with tf.device('/cpu:0'):
    preds = 0.5 * (output_0 + output_1)

# we only run the `preds` tensor, so that only the two
# replicas on GPU get run (plus the merge op on CPU)
output_value = sess.run([preds], feed_dict={x: data})

NameError: name 'data' is not defined

In [31]:
server = tf.train.Server.create_local_server()
sess = tf.Session(server.target)

from keras import backend as K
K.set_session(sess)

# Export a model with TensorFlow-serving

In [33]:
from keras import backend as K

K.set_learning_phase(0)  # all new operations will be in test mode from now on

# serialize the model and get its weights, for quick re-building
config = previous_model.get_config()
weights = previous_model.get_weights()

# re-build a model where the learning phase is now hard-coded to 0
from keras.models import model_from_config
new_model = model_from_config(config)
new_model.set_weights(weights)

from tensorflow_serving.session_bundle import exporter

export_path = ... # where to save the exported graph
export_version = ... # version number (integer)

saver = tf.train.Saver(sharded=True)
model_exporter = exporter.Exporter(saver)
signature = exporter.classification_signature(input_tensor=model.input,
                                              scores_tensor=model.output)
model_exporter.init(sess.graph.as_graph_def(),
                    default_graph_signature=signature)
model_exporter.export(export_path, tf.constant(export_version), sess)

NameError: name 'previous_model' is not defined